In [3]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
import os

In [5]:
# Create a connection to the SQLite database
engine = create_engine('sqlite:///data/churn_data2.db')
# Query the ‘churn’ table
data_from_db = pd.read_sql('SELECT * FROM churn', engine)
# Display the first few rows of the queried data
print(data_from_db.head())

   customerID  SeniorCitizen    tenure  MonthlyCharges  TotalCharges Churn  \
0  7590-VHVEG              0 -1.277354       -1.160241     -0.994171    No   
1  5575-GNVDE              0  0.066323       -0.259611     -0.173232    No   
2  3668-QPYBK              0 -1.236636       -0.362635     -0.959606   Yes   
3  7795-CFOCW              0  0.514215       -0.746482     -0.194752    No   
4  9237-HQITU              0 -1.236636        0.197351     -0.940403   Yes   

   gender_Male  Partner_Yes  Dependents_Yes  PhoneService_Yes  ...  \
0            0            1               0                 0  ...   
1            1            0               0                 1  ...   
2            1            0               0                 1  ...   
3            1            0               0                 0  ...   
4            0            0               0                 1  ...   

   StreamingTV_No internet service  StreamingTV_Yes  \
0                                0                0   


In [11]:
from sklearn.preprocessing import LabelEncoder

# Fill missing values
data_from_db.fillna(method='ffill', inplace=True)

# Encode categorical variables
label_encoders = {}
for column in data_from_db.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    data_from_db[column] = label_encoders[column].fit_transform(data_from_db[column])

# Display the first few rows after preprocessing
data_from_db.head()


/var/folders/8b/7m7r6x_s09n0br47ncq0bt000000gn/T/ipykernel_11307/2190014093.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_from_db.fillna(method='ffill', inplace=True)


,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,5375,0,-1.277354,-1.160241,-0.994171,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,3962,0,0.066323,-0.259611,-0.173232,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
2,2564,0,-1.236636,-0.362635,-0.959606,1,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
3,5535,0,0.514215,-0.746482,-0.194752,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,6511,0,-1.236636,0.197351,-0.940403,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0


In [12]:
#split data into testing and training bins
from sklearn.model_selection import train_test_split

# Define features and target
target_col = 'Churn'  # Change to your target column name
X = data_from_db.drop(columns=[target_col])
y = data_from_db[target_col]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Initialize the model
model_type = 'classification'

if model_type == 'classification':
    model = RandomForestClassifier(n_estimators=100, random_state=42)
else:
    model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [21]:
from sklearn.metrics import accuracy_score, r2_score

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
if model_type == 'classification':
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    if accuracy >= 0.75:
        print("Model accuracy >= 75%")
    else:
        print("Model accuracy < 75%")

else:
    r_squared = r2_score(y_test, y_pred)
    print(f"R-squared: {r_squared}")
    if r_squared >= 0.80:
        print("Model has R-squared >= 0.80")
    else:
        print("Model has R-squared < 0.80")

Accuracy: 0.7934705464868701
Model accuracy >= 75%


In [22]:
# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")
if r_squared >= 0.80:
    print("Model has R-squared >= 0.80")
else:
    print("Model has R-squared < 0.80")

R-squared: -0.061048888796877865
Model has R-squared < 0.80


In [25]:
accuracy_percent = accuracy * 100
accuracy_percent

79.347054648687